In [29]:
# Import libraries for data manipulation and ETL pipeline
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pandarallel import pandarallel
pandarallel.initialize(verbose=0)
from tqdm import tqdm

In [30]:
# Establish API connection
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

cid = '3fda75b7146a4769b207ee44017b3abe'
secret = '2a755cb04a18406b9394dbef2f8069dd'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [31]:
# Connect to SQL Server
import sqlalchemy as sql
from sqlalchemy import text
import pyodbc
server = 'LAPTOP-V3754MEK' 
database = 'Spotify'

engine = sql.create_engine('mssql+pyodbc://' + server + '/' + database + '?trusted_connection=yes&driver=ODBC+Driver+13+for+SQL+Server')
conn = engine.connect()
results = engine.execute("SELECT track_uri2 FROM dim_tracks ORDER BY track_uri2")
results = pd.Series(results)
results.head()

0    (spotify:track:0002yNGLtYSYtc0X6ZnFvp)
1    (spotify:track:00039MgrmLoIzSpuYKurn9)
2    (spotify:track:0003Z98F6hUq7XxqSRM87H)
3    (spotify:track:0004ExljAge0P5XWn1LXmW)
4    (spotify:track:0005rgjsSeVLp1cze57jIN)
dtype: object

In [32]:
# Extract tracks from API
output_df = pd.DataFrame()
for row in tqdm(results[750001:760001]):
      output = pd.DataFrame(sp.audio_features(row))
      output_df = pd.concat([output_df, output])
output_df.head()

100%|██████████| 10000/10000 [14:07<00:00, 11.80it/s] 


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.706,0.607,9,-6.797,0,0.0255,0.07810,0.001100,0.2690,0.795,105.986,audio_features,2MtMNJhyfiEvP96Wo5TZMr,spotify:track:2MtMNJhyfiEvP96Wo5TZMr,https://api.spotify.com/v1/tracks/2MtMNJhyfiEv...,https://api.spotify.com/v1/audio-analysis/2MtM...,329307,4
0,0.622,0.984,10,-1.531,0,0.0779,0.00162,0.021300,0.6960,0.666,108.007,audio_features,2mTmPKk4QvLsQHHzluLoYZ,spotify:track:2mTmPKk4QvLsQHHzluLoYZ,https://api.spotify.com/v1/tracks/2mTmPKk4QvLs...,https://api.spotify.com/v1/audio-analysis/2mTm...,220900,4
0,0.726,0.607,11,-9.177,0,0.0506,0.12300,0.000892,0.1080,0.225,127.991,audio_features,2mtmVxxieIIJdMqDvy5aq3,spotify:track:2mtmVxxieIIJdMqDvy5aq3,https://api.spotify.com/v1/tracks/2mtmVxxieIIJ...,https://api.spotify.com/v1/audio-analysis/2mtm...,204375,4
0,0.497,0.777,7,-6.469,1,0.2200,0.20400,0.003810,0.0928,0.594,77.048,audio_features,2mTMyXn3knPor0BK68okuE,spotify:track:2mTMyXn3knPor0BK68okuE,https://api.spotify.com/v1/tracks/2mTMyXn3knPo...,https://api.spotify.com/v1/audio-analysis/2mTM...,172493,5
0,0.432,0.490,0,-5.924,1,0.0457,0.44600,0.000000,0.3400,0.538,159.225,audio_features,2mtmzpVrpTRuS3yuL8iFz4,spotify:track:2mtmzpVrpTRuS3yuL8iFz4,https://api.spotify.com/v1/tracks/2mtmzpVrpTRu...,https://api.spotify.com/v1/audio-analysis/2mtm...,186000,4


In [33]:
# Load data into SQL Server
output_df = output_df[['uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
output_df = output_df.rename(columns={'key':'music_key', 'mode':'modal'})
output_df.to_sql("audio_features2", engine, if_exists='append', index=False)
print(f'{len(output_df)} rows imported to database.')
new_query = engine.execute('SELECT COUNT(*) FROM audio_features2')
new_query = pd.Series(new_query)
print(f'App contains {new_query[0]} songs.')

10000 rows imported to database.
App contains (750001,) songs.
